In [13]:
import sys
sys.path.append('../')
from src.data_input_preparation import data_input_preparation

In [5]:
data = data_input_preparation()

/usr/local/lib/python2.7/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/usr/local/lib/python2.7/dist-packages/scipy/signal/signaltools.py:3463: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return y[sl]


*******************************************
Data saving string:
../files/data/dataGMCA_decf16_J_2_norm_1_bsize_2048_nobs_250
*******************************************


In [2]:
import sys
sys.path.append('../')
import numpy as np
from src.utils2 import divisorGenerator
from src.utils2 import Make_Experiment_GG
from src.misc_bgmca2 import *
from src.DGMCA import DGMCA as dgmca
from src.GMCA import GMCA as gmca
from src.data_input_preparation import data_input_preparation
import copy as cp
from tqdm import tqdm
import time

input_data_path = '../files/input_data/'
data_path = '../files/data/'
plots_path = '../files/plots/'


Jdec = 2
dec_factor = 64
batch_size = 1024
n_s = 5 # Number of sources
n_obs = 250 # Number of observations
SNR_level = 60.0
normOpt = 1
alpha_init = 2. # Alpha parameter for the thresholding strategy
numIts = 1

data_DGMCA = data_input_preparation(Jdec=Jdec, dec_factor=dec_factor, batch_size=batch_size, n_obs=n_obs, SNR_level=SNR_level, normOpt=normOpt,\
    input_data_path=input_data_path, output_data_path=data_path)


/usr/local/lib/python2.7/dist-packages/scipy/signal/filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
/usr/local/lib/python2.7/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/usr/local/lib/python2.7/dist-packages/scipy/signal/signaltools.py:3463: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return y[sl]


*******************************************
Data saving string:
../files/data/dataGMCA_decf64_J_2_norm_1_bsize_1024_nobs_250
*******************************************


In [3]:
# Extract the variables
X = data_DGMCA['X']
A0 = data_DGMCA['A0']
S0 = data_DGMCA['S0']
X_GMCA = data_DGMCA['X_GMCA']
S0_GMCA = data_DGMCA['S0_GMCA']
sizeBlock = batch_size
n_s = data_DGMCA['n_s'][0,0]
WNFactors = data_DGMCA['WNFactors'][0]
X0_GMCA = data_DGMCA['X0']
normOpt = data_DGMCA['normOpt'] 
totalSize = X.shape[1] 

TypeError: 'int' object has no attribute '__getitem__'

In [8]:
data_DGMCA['WNFactors']

array([4.16470412e-08, 5.05267988e-09, 5.49415421e-09, 3.55299056e-09,
       1.84950920e-09])